In [ ]:
!pip install -q datasets[audio] peft transformers torch librosa evaluate

from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.6 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from datasets import load_dataset, concatenate_datasets
from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
from peft import LoraConfig, get_peft_model

In [ ]:
model_id = "small"
language = "ne"
task = "transcribe"
model_name_or_path = f"openai/whisper-{model_id}"

processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)
model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path)
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

In [ ]:
peft_config = LoraConfig(
    task_type="SEQ_2_SEQ_LM",
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    modules_to_save=["lm_head"]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 2,654,208 || all params: 244,389,120 || trainable%: 1.0861


In [ ]:
# Example: Nepali dataset (replace with your own)
nepali = load_dataset(
    "amitpant7/nepali-speech-to-text",
    split="train.0"
)
# English dataset for replay
english = load_dataset("librispeech_asr", "clean", split="train.100[:2%]")

def prepare_dataset(batch):
    # Load & process audio
    audio = batch["audio"]
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # Tokenize transcription
    # Use "text" instead of "transcription" for librispeech
    batch["labels"] = processor.tokenizer(batch["text"]).input_ids
    return batch

nepali = nepali.map(prepare_dataset, remove_columns=nepali.column_names, num_proc=1)
english = english.map(prepare_dataset, remove_columns=english.column_names, num_proc=1)

# Add language flag (for optional distillation later)
nepali = nepali.add_column("is_english", [0]*len(nepali))
english = english.add_column("is_english", [1]*len(english))

# Mix Nepali and English (10% English)
mix_ratio = 0.1
mixed_train = concatenate_datasets([
    nepali,
    english.shuffle(seed=42).select(range(int(len(nepali)*mix_ratio)))
]).shuffle(seed=42)

print(f"Total mixed training samples: {len(mixed_train)}")

README.md: 0.00B [00:00, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


data/train.0-00000-of-00001.parquet:   0%|          | 0.00/192M [00:00<?, ?B/s]

data/train.1-00000-of-00001.parquet:   0%|          | 0.00/469M [00:00<?, ?B/s]

data/train.2-00000-of-00001.parquet:   0%|          | 0.00/287M [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/2620 [00:00<?, ? examples/s]

Generating train.100 split:   0%|          | 0/28539 [00:00<?, ? examples/s]

Generating train.360 split:   0%|          | 0/104014 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2703 [00:00<?, ? examples/s]

ValueError: Unknown split "train.clean.100". Should be one of ['test', 'train.100', 'train.360', 'validation'].